# Recurrent Neural Networks: Programming Practice

COSC 410: Applied Machine Learning\
Colgate University\
*Prof. Apthorpe*

## Overview

This notebook will give you practice with the following topics:
  1. Creating and training simple RNNs
  2. Creating and training LSTM & GRU networks

We will be using a new **natural language** dataset of IMDB movie reviews. We will attempt to perform a classification task to label reviews as having either *positive* or *negative* sentiment. The reviews have already been preprocessed from words --> numbers, such that the most common word is encoded as 1, the second most common word is encoded as 2, etc. 

## Part 1. Data Import & Inspection

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as ks

In [ ]:
max_features = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review

(x_train, y_train), (x_val, y_val) = ks.datasets.imdb.load_data(num_words=max_features) # Load the dataset

x_train = ks.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen) # Pad reviews with 0s to equal length
x_val = ks.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen) # Pad reviews with 0s to equal length

print(len(x_train), "Training sequences") # Print number of training reviews
print(len(x_val), "Validation sequences") # Print number of validation reviews

print(np.unique(y_train, return_counts=True)[1]) # print counts of classes in training set
print(np.unique(y_val, return_counts=True)[1]) # print counts of classes in validation set

**Note:** One of the benefits of RNNs is that they can handle sequences of differing lengths, so why do we pad the training and validation data to equal lengths? ... Training time! Keras is optimized for mini-batch gradient descent when the training examples are non-ragged matrices (a.k.a. *tensors*...hence "Tensorflow"). Once we have trained the model, we can perform predictions on *new* reviews of arbitary length.  

In [ ]:
# Function that converts an embedded review back into English bag-of-words representation
def reverse_embedding(review):
    words_to_ints = ks.datasets.imdb.get_word_index(path="imdb_word_index.json")
    reverse_embedding_map = dict(zip(words_to_ints.values(), words_to_ints.keys()))
    return " ".join([reverse_embedding_map.get(x, "") for x in review])

# Function that prints the label (1 = positive review, 0 = negative review) and bag-of-words text of a review
def print_example_and_label(idx):
    print(f"Label: {y_train[idx]}")
    print(reverse_embedding(x_train[idx]) + "\n")
    
print_example_and_label(0)
print_example_and_label(1)

## Part 2. 1-Node RNNs

We'll start by creating three different 1-Node RNNs with `SimpleRNN`, `LSTM`, and `GRU` nodes. 

In [ ]:
# SimpleRNN model
model = ks.Sequential([
    ks.Input(shape=(None,)),                     # Input for variable-length sequences
    ks.layers.Embedding(max_features, 1),        # Embed each integer in a 1-dimensional vector
    ks.layers.SimpleRNN(1, activation="sigmoid") # Single RNN node     
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

In [ ]:
# LSTM model
model = ks.Sequential([
    ks.Input(shape=(None,)),                   # Input for variable-length sequences
    ks.layers.Embedding(max_features, 1),      # Embed each integer in a 1-dimensional vector
    ks.layers.LSTM(1, activation="sigmoid")    # Single LSTM node 
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=15, validation_data=(x_val, y_val))

In [ ]:
# GRU model
model = ks.Sequential([
    ks.Input(shape=(None,)),                   # Input for variable-length sequences
    ks.layers.Embedding(max_features, 1),      # Embed each integer in a 64-dimensional vector
    ks.layers.GRU(1, activation="sigmoid")     # Single GRU node 
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=15, validation_data=(x_val, y_val))

## Part 3. Recurrent Layer Hyperparameters

`SimpleRNN`, `LSTM` and `GRU` layers share several hyperparameters that can be set via keyword arguments. The documentation for each of these layers is here: https://keras.io/api/layers/recurrent_layers/
  * `return_sequences`: Whether the layers should return a single value for each input sequence (`False`, *default*) or a new multi-step sequence for each input sequence (`True`)
  * `kernel_regularizer`, `bias_regularizer`, and `recurrent_regularizer` for L1, L2, or L1L2 regularization
  * `dropout` and `recurrent_dropout` for setting dropout rates of forward and recurrent connections. 

## Part 4: Stacked RNNs

Finally, let's try a stacked LSTM network with 2 hidden layers. 

In [ ]:
# Stacked LSTM model
model = ks.Sequential([
    ks.Input(shape=(None,)),                    # Input for variable-length sequences
    ks.layers.Embedding(max_features, 1),      # Embed each integer in a 64-dimensional vector
    ks.layers.LSTM(64, return_sequences=True),  # Add recurrent layers
    ks.layers.LSTM(64, return_sequences=True), 
    ks.layers.LSTM(1, activation="sigmoid")
])
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val))

We can see that the stacked RNN has higher plateau performance and reaches that performance more quickly. However, the increasing difference between train accuracy and val accuracy indicates that we are in an overfitting regime, so it doesn't make sense to increase the size of the network further. 